# ETL 5 Pair 25/08
## ETL Carga I

Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.

### Imports

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import mysql.connector

from datetime import datetime, timedelta

### Datos

In [2]:
df_energia_ccaa = pd.read_pickle("data/pairETL3_energia_ccaa.pkl")
df_energia_ccaa.head(2)

,value,percentage,tipo_energia,comunidad,id_comunidad,fecha
0,460.05,1.0,No renovable,Ceuta,8744,2011-01-01
1,462.98,1.0,No renovable,Ceuta,8744,2011-01-02


In [3]:
df_energia_nacional = pd.read_pickle("data/pairETL3_energia_nacional.pkl")
df_energia_nacional.head(2)

,value,percentage,tipo_energia,fecha
0,182024.98,0.29,Renovable,2011-01-01
1,265705.89,0.39,Renovable,2011-01-02


In [4]:
df_poblacion_ccaa = pd.read_pickle("data/pairETL3_poblacion_comunidades.pkl")
df_poblacion_ccaa.head(2)

,Unnamed: 0,Comunidad,indice,Año,Total
0,0,Ceuta,8744,2022,83.117
1,1,Ceuta,8744,2021,83.517


### Clases

In [5]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña

   
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La BBDD ya existe")
            
      
    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            

    def check_comunidades(self):
    
        mydb = mysql.connector.connect(user='root',
                                      password=f"{self.contraseña}",
                                      host='127.0.0.1',
                                      database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        # query para extraer los valores únicos de ciudades de la tabla de localidades 
        query_existe_ciudad = f"""
                SELECT DISTINCT comunidades FROM comunidades
                """
        mycursor.execute(query_existe_ciudad)
        comunidades = mycursor.fetchall()
        return comunidades
    
    # método para sacar el id de una ciudad en concreto 
    def sacar_id_comunidad(self, comunidad):
        
        mydb = mysql.connector.connect(user='root',
                                       password= f'{self.contraseña}',
                                       host='127.0.0.1', 
                                       database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()
        
        try:
            query_sacar_id = f"SELECT idcomunidades FROM comunidades WHERE comunidades = '{comunidad}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
            return "Sorry, no tenemos esa comunidad en la BBDD y por lo tanto no te podemos dar su id. "
     
    
    def sacar_id_fecha(self, fecha):
        mydb = mysql.connector.connect(user='root', password=f'{self.contraseña}',
                                          host='127.0.0.1', database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        try:
            query_sacar_id = f"SELECT idfechas FROM fechas WHERE fecha = '{fecha}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
             return "Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. "
            

### Creación de la BBDD

In [6]:
carga = Cargar("energia2", "AlumnaAdalab")
carga

In [7]:
carga.crear_bbdd()

Conexión realizada con éxito


### Creación de las tablas

In [8]:
tabla_fechas = '''
CREATE TABLE IF NOT EXISTS `energia2`.`fechas` (
  `idfechas` INT NOT NULL AUTO_INCREMENT,
  `fecha` DATE NULL,
  PRIMARY KEY (`idfechas`))
ENGINE = InnoDB;
'''

In [9]:
tabla_nacional_renovable_no_renovable = '''
CREATE TABLE IF NOT EXISTS `energia2`.`nacional_renovable_no_renovable` (
  `idnacional_renovable_no_renovable` INT NOT NULL AUTO_INCREMENT,
  `porcentaje` INT NULL,
  `tipo_energia` VARCHAR(45) NULL,
  `valor` DECIMAL NULL,
  `fechas_idfechas` INT NOT NULL,
  PRIMARY KEY (`idnacional_renovable_no_renovable`),
  INDEX `fk_nacional_renovable_no_renovable_fechas_idx` (`fechas_idfechas` ASC) VISIBLE,
  CONSTRAINT `fk_nacional_renovable_no_renovable_fechas`
    FOREIGN KEY (`fechas_idfechas`)
    REFERENCES `energia2`.`fechas` (`idfechas`))
ENGINE = InnoDB;
'''

In [10]:
tabla_comunidades = '''
CREATE TABLE IF NOT EXISTS `energia2`.`comunidades` (
  `idcomunidades` INT NOT NULL AUTO_INCREMENT,
  `comunidades` VARCHAR(45) NULL,
  PRIMARY KEY (`idcomunidades`))
ENGINE = InnoDB;
'''

In [11]:
tabla_comunidades_renovable_no_renovable = '''
CREATE TABLE IF NOT EXISTS `energia2`.`comunidades_renovable_no_renovable` (
  `idcomunidades_renovable_no_renovable` INT NOT NULL AUTO_INCREMENT,
  `porcentaje` INT NULL,
  `tipo_energia` VARCHAR(45) NULL,
  `valor` DECIMAL NULL,
  `fechas_idfechas` INT NOT NULL,
  `comunidades_idcomunidades` INT NOT NULL,
  PRIMARY KEY (`idcomunidades_renovable_no_renovable`),
  INDEX `fk_comunidades_renovable_no_renovable_fechas1_idx` (`fechas_idfechas` ASC) VISIBLE,
  INDEX `fk_comunidades_renovable_no_renovable_comunidades1_idx` (`comunidades_idcomunidades` ASC) VISIBLE,
  CONSTRAINT `fk_comunidades_renovable_no_renovable_fechas1`
    FOREIGN KEY (`fechas_idfechas`)
    REFERENCES `energia2`.`fechas` (`idfechas`),
  CONSTRAINT `fk_comunidades_renovable_no_renovable_comunidades1`
    FOREIGN KEY (`comunidades_idcomunidades`)
    REFERENCES `energia2`.`comunidades` (`idcomunidades`))
ENGINE = InnoDB;
'''

In [12]:
carga.crear_insertar_tabla(tabla_fechas)
carga.crear_insertar_tabla(tabla_nacional_renovable_no_renovable)
carga.crear_insertar_tabla(tabla_comunidades)
carga.crear_insertar_tabla(tabla_comunidades_renovable_no_renovable)

### Validación: No hay datos insertados

In [13]:
id_comunidad = carga.sacar_id_comunidad("Madrid")
id_comunidad

'Sorry, no tenemos esa comunidad en la BBDD y por lo tanto no te podemos dar su id. '

In [14]:
id_fecha = carga.sacar_id_fecha(2011-11-10)
id_fecha

'Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. '

### Inserción de datos en la tabla 'fechas'

Como puede haber fechas que no existan en el dataframe 'df_energia_ccaa' o en el dataframe 'df_energia_nacional', se van a seguir los siguientes pasos:

1. Unir ambos dataframes por la columna 'fecha'.
2. Como posiblemente, haya fechas duplicadas, eliminamos los posibles duplicados.
3. Insertamos en la tabla 'fechas' los registros únicos de la columna 'fecha' de nuestro nuevo dataframe.

In [15]:
df_fechas = pd.concat([df_energia_ccaa['fecha'], df_energia_nacional['fecha']], axis = 0, join = "inner", ignore_index = True)
df_fechas = pd.DataFrame(df_fechas)
df_fechas

,fecha
0,2011-01-01
1,2011-01-02
2,2011-01-03
3,2011-01-04
4,2011-01-05
...,...
43754,2022-12-27
43755,2022-12-28
43756,2022-12-29
43757,2022-12-30


In [16]:
df_fechas.duplicated().sum()

39376

In [17]:
df_fechas = df_fechas.drop_duplicates()
df_fechas

,fecha
0,2011-01-01
1,2011-01-02
2,2011-01-03
3,2011-01-04
4,2011-01-05
...,...
32438,2022-12-27
32439,2022-12-28
32440,2022-12-29
32441,2022-12-30


In [18]:
df_fechas.duplicated().sum()

0

In [19]:
for indice, fila in df_fechas.iterrows(): 
    
    query_fechas = f"""
            INSERT INTO fechas (fecha) 
            VALUES ("{fila['fecha']}");
            """
    carga.crear_insertar_tabla(query_fechas)

### Inserción de datos en la tabla 'comunidades'

Los dataframe que podemos utilizar son 'df_energia_ccaa' y  'df_poblacion_ccaa'. En este caso, vamos a utilizar el dataframe 'df_energia_ccaa' porque así tendremos los mismos nombres que vamos a tener que enlazar cuando hagamos la inserción de los datos de energia de las CCAA.

1. Vamos a crear un nuevo dataframe con los registros de la columna 'comunidad'
2. Eliminamos los posibles duplicados del nuevo dataframe.
3. Insertamos en la tabla 'comunidades' los registros únicos de la columna 'comunidad' de nuestro nuevo dataframe.

In [20]:
df_comunidades = df_energia_ccaa[['comunidad']]
df_comunidades.sample()

,comunidad
16399,Illes Balears


In [21]:
df_comunidades.duplicated().sum()

34974

In [22]:
df_comunidades = df_comunidades.drop_duplicates()

In [23]:
df_comunidades.duplicated().sum()

0

In [24]:
df_comunidades

,comunidad
0,Ceuta
365,Melilla
1095,Andalucía
1119,Aragón
1143,Cantabria
1167,Castilla - La Mancha
1191,Castilla y León
1215,Cataluña
1239,País Vasco
1263,Principado de Asturias


In [25]:
for indice, fila in df_comunidades.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_comunidad = f"""
                INSERT INTO comunidades (comunidades) 
                VALUES ("{fila['comunidad']}");
                """
    # llamamos a la funcion check_ciudades para saber que ciudades tenemos en la BBDD
    comunidades = carga.check_comunidades()
    
    # Si la BBDD de datos esta vacía,  podemos insertar la ciudad. 
    # 
    if len(comunidades) == 0 or fila['comunidad'] not in comunidades[0]: 
        carga.crear_insertar_tabla( query_comunidad)

    else:
        print(f"{fila['comunidades']} ya esta en nuestra BBDD")

### Inserción de los datos en la tabla 'comunidades_renovable_no_renovable'

El dataframe que vamos a utilizar es 'df_energia_ccaa'.

1. Insertamos en la tabla 'comunidades_renovable_no_renovable' los registros del dataframe, buscando los ids correspondientes a la columna 'fecha' y 'comunidad' para poder enlazar correctamente la información de la BBDD.

In [26]:
df_energia_ccaa.head(2)

,value,percentage,tipo_energia,comunidad,id_comunidad,fecha
0,460.05,1.0,No renovable,Ceuta,8744,2011-01-01
1,462.98,1.0,No renovable,Ceuta,8744,2011-01-02


In [ ]:
for indice, fila in df_energia_ccaa.iterrows():
    
    id_comunidad = carga.sacar_id_comunidad(fila['comunidad'])
    id_fecha = carga.sacar_id_fecha(fila["fecha"])
        
    query_valores_ccaa = f"""
                INSERT INTO comunidades_renovable_no_renovable (porcentaje, tipo_energia, valor, fechas_idfechas, comunidades_idcomunidades) 
                VALUES ({fila['percentage']}, "{fila['tipo_energia']}", {fila['value']}, {id_fecha}, {id_comunidad});
                """
    carga.crear_insertar_tabla(query_valores_ccaa)
    

### Inserción de los datos en la tabla 'nacional_renovable_no_renovable'

El dataframe que vamos a utilizar es 'df_energia_nacional'.

1. Insertamos en la tabla 'nacional_renovable_no_renovable' los registros del dataframe, buscando los ids correspondientes a la columna 'fecha' para poder enlazar correctamente la información de la BBDD.

In [27]:
df_energia_nacional.head(2)

,value,percentage,tipo_energia,fecha
0,182024.98,0.29,Renovable,2011-01-01
1,265705.89,0.39,Renovable,2011-01-02


In [ ]:
for indice, fila in df_energia_nacional.iterrows():
    
    id_fecha = carga.sacar_id_fecha(fila["fecha"])
    
    query_valores_nacional = f"""
                INSERT INTO nacional_renovable_no_renovable (porcentaje, tipo_energia, valor, fechas_idfechas)
                VALUES ({fila['percentage']}, "{fila['tipo_energia']}", {fila['value']}, {id_fecha});
                """
    carga.crear_insertar_tabla(query_valores_nacional)